In [2]:
!pip install tqdm

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------------- ------------------------ 20.5/57.6 kB 330.3 kB/s eta 0:00:01
     --------------------------- ---------- 41.0/57.6 kB 393.8 kB/s eta 0:00:01
     --------------------------------- ---- 51.2/57.6 kB 327.7 kB/s eta 0:00:01
     -------------------------------------- 57.6/57.6 kB 378.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/78.3 kB ? eta -:--:--
   ---------------------------------------- 78.3/78.3 kB 4.5 MB/s eta 0:00:00


In [3]:
import random
import re
from time import sleep

import pandas as pd
from selenium import webdriver

import tqdm

In [4]:
def obtener_lista_actores_unicos(lista_ficheros_csv, columna_actor):
    
    # Lista para almacenar todos los DataFrames
    total = []

    # Iterar sobre los archivos y cargarlos en DataFrames
    for fichero in lista_ficheros_csv:
        df = pd.read_csv(fichero, index_col=0)
        total.append(df)

    # Concatenar todos los DataFrames en uno solo
    df_concatenado = pd.concat(total)
    
    # Eliminar duplicados
    df_concatenado = df_concatenado.drop_duplicates(subset=[columna_actor])
    print(df_concatenado.head())

    # devolver lista de actores
    return list(df_concatenado[columna_actor])

In [5]:
def guardar_resultados(lista_datos, fichero_csv):
    df = pd.DataFrame(lista_datos)
    df.columns = ["actor", "año nacimiento", "que hace", "conocido por", "lista premios"]
    df.to_csv(fichero_csv)

In [6]:
def obtener_informacion_actores(lista_actores, guardar_en_csv):
    
    # cargar chrome en español
    options = webdriver.ChromeOptions()
    options.add_argument("--accept-lang=es")
    driver = webdriver.Chrome(options=options)
    driver.maximize_window()

    # variable para los descansos
    proximo_descanso = random.randint(50,200)
    
    #Definimos una lista para guardar los resultados 
    resultados = []
    
    # Iterar sobre los actores del DataFrame
    for indice, actor in enumerate(tqdm.tqdm(lista_actores)):
        #Hace cada vez que itera 200 actores 
        if indice % 200 == 0 and indice > 0: 
            print(f"Guardando copia parcial de actores en {guardar_en_csv}. Van {indice + 1} actores.")
            guardar_resultados(resultados, guardar_en_csv)
            
        # descanso de vez en cuando
        if indice > proximo_descanso:
            print("Descansando un poco")
            proximo_descanso = indice + random.randint(50,200)
            sleep(random.uniform(10,30))
        try:
            #obtener información del actor de la funcion obtener_informacion_actor 
            info_actor = obtener_informacion_actor(driver, actor)
            resultados.append(info_actor)
        except:
            pass
        
    print(f"Guardando la información de los actores en {guardar_en_csv}")
    guardar_resultados(resultados, guardar_en_csv)
    return resultados


In [7]:
def obtener_informacion_actor(driver, actor):
    # Abre la página de IMDB para buscar al actor
    driver.get(f"https://www.imdb.com/search/name/?name={actor}")
    sleep(random.uniform(1.0, 2.5))

    # Click en el primer resultado
    driver.find_element("tag name","h3").click()
    sleep(random.uniform(1.0, 2.5))

    # Obtener información sobre el actor
    try:
        que_hace = driver.find_element('xpath',
            '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
        que_hace =  que_hace.split("\n")
    except:
        que_hace = ["Reparto"]
    allweb = driver.find_element("xpath","/html/body").text
    try:
        ano_nacimiento = re.compile(r"\d{1,2} de \w+ de (\d{4})").findall(allweb)[0]
    except:
        ano_nacimiento = None

    # Click en la biografía
    driver.find_element('class name',"ipc-html-content-inner-div").click()
    sleep(random.uniform(1.0, 2.5))

    # Obtener enlaces de películas conocidas
    bio = driver.find_element('xpath', '//*[@id="__next"]/main/div/section/div/section/div/div[1]/section[2]/div[2]')
    try:
        enlaces = [x.text for x in bio.find_elements("class name", "ipc-md-link")]
        re_pelicula_ano = r"([\w|\s]+)\s\(\d{4}\)"
        conocido_por = [re.match(re_pelicula_ano, enlace).group(0) for enlace in enlaces if
                        re.match(re_pelicula_ano, enlace)]
    except:
        conocido_por = None

    try:
        # Click en la sección de premios
        driver.find_element('xpath',
            '//*[@id="__next"]/main/div/section/section/div[3]/section/section/div[1]/div/div/div[2]/ul/li[2]/a').click()
        sleep(random.uniform(1.5, 3.0))
    
        # Obtener premios ganados
        premios = driver.find_elements('tag name',"section")[6:]
        lista_premios = [premio.text.split("\n")[0] for premio in premios if "Ganador" in premio.text] 
    except:
        lista_premios = None

    # Guardar información en la lista de resultados
    return (actor, ano_nacimiento, que_hace, conocido_por, lista_premios)

In [9]:
# Uso de la función
lista_ficheros =["Drama\\Actores\\actores_cortos_drama_2000_2004.csv", "Drama\\Actores\\actores_pelis_drama_2000_2004.csv"]
columna = "actores" 
fichero_donde_guardar = "informacion_actores.csv"

lista_actores = obtener_lista_actores_unicos(lista_ficheros, columna)
datos_actores = obtener_informacion_actores(lista_actores, fichero_donde_guardar)
# datos_actores contiene los datos de los actores, aunque ya deberían haber sido guardados en el fichero_donde_guardar

          actores
0  Juana Migliore
1  Amy K. Cropper
2   Robert Clarke
3  Cailey Scherer
4      Paul Amend


  0%|          | 66/28182 [13:16<76:18:26,  9.77s/it] 

Descansando un poco


  0%|          | 107/28182 [13:32<59:12:18,  7.59s/it]


KeyboardInterrupt: 

In [17]:
#libreria que sirve para coger todos los archivos de la dirección facilitada de manera automática 
import glob

base_dir = "../../../proyecto_2/Adalab-proyecto-da-promo-G-modulo-2-team-1"
# !ls {base_dir}
#Encuentra todos los ficheros que cumplen el siguiente patron
lista_ficheros = glob.glob(f'{base_dir}/**/actores*.csv')
columna = "0"  # o " actores"
fichero_donde_guardar = "informacion_actores.csv"

In [18]:
lista_actores = obtener_lista_actores_unicos(lista_ficheros, columna)
print(len(lista_actores))
lista_actores

                   0
0      Kees Scholten
1    Michael O'Neill
2         Emily Woof
3  Davide Dragonetti
4      Tracey Walter
56690


['Kees Scholten',
 "Michael O'Neill",
 'Emily Woof',
 'Davide Dragonetti',
 'Tracey Walter',
 'Judith Kaplan',
 'Antoine Chappey',
 'Luciano Virieux',
 'Tom Wisdom',
 'Jackie Martling',
 'Laura Cerón',
 'Chad Lindberg',
 'Kim Murphy',
 'Cristos',
 'Carlos Julio Molina',
 'Mikhail Krylov',
 'Mitre',
 'Lyudmila Aleksandrova',
 'Eva Fisher',
 'Jorge Richter',
 'Stelio Savante',
 'Rona Hartner',
 'Lucia',
 'Gary Cruz',
 'Silvester Bozic',
 'Valeri Ross',
 'Philippe Harel',
 'Sofia Bertolotto',
 'Tara Ellis',
 'Brendan Fletcher',
 'Valter Skarsgård',
 'Andreas Beckett',
 'Bastian Trost',
 'Scott Caan',
 'Jennifer McComb',
 'Maria Emília Correia',
 'Janan Raouf',
 'Barry Papick',
 'Jeanne Tripplehorn',
 'Mark Christopher Collins',
 'Bharat Dabholkar',
 'Leslie France',
 'Angéline Tischi',
 'Pierre Bokma',
 'Laurence De Greef',
 'Zdenek Rauser',
 'Rob Mallard',
 'Susanne Walbaum',
 'Thomas Jay Ryan',
 'Björn Kjellman',
 'Luchisha Evans',
 'Moritz Alioto',
 'Chris Fisher',
 'Valérie Stroh',
 '

In [33]:
datos_actores = obtener_informacion_actores(lista_actores[:200], fichero_donde_guardar)

 32%|███▎      | 65/200 [15:10<33:36, 14.94s/it]  

Descansando un poco


 73%|███████▎  | 146/200 [34:08<12:30, 13.89s/it]

Descansando un poco


100%|██████████| 200/200 [48:17<00:00, 14.49s/it]

Guardando la información de los actores en informacion_actores.csv


In [23]:
len(datos_actores)

10